# 03 - EDA

In [1]:
import sys
sys.executable

'/usr/local/bin/python'

## Imports

In [2]:
import pandas as pd

import constants as cnst
import stock_utils as su

pd.set_option('display.max_columns', None)

NSE_DATA_DIR = PosixPath('../data/NSE') | Valid: True
PROCESSED_DATA_DIR = PosixPath('../data/processed') | Valid: True


## Constants

In [3]:
stock_symbols = su.get_all_stock_symbols(
    cnst.NSE_DATA_DIR
)

stock_symbols

['HDFCBANK', 'ITBEES']

In [4]:
STOCK_SYMBOL = stock_symbols[0]
STOCK_SYMBOL

'HDFCBANK'

## Data loading

In [5]:
stock_df = pd.read_parquet(
    cnst.PROCESSED_DATA_DIR.joinpath(f'{STOCK_SYMBOL}-processed.parquet')
)

stock_df

,Date,Open,High,Low,LTP,Close,VWAP,52W H,52W L,Volume,Value,#Trades,IsGreen,Is52WLow,Is52WHigh,Day,Month,Year,Weekday,DayOfYear,Quarter,DaysSinceLastTradingSession,Close 3MA,Close 7MA,Close 15MA,Close 30MA,VWAP 3MA,VWAP 7MA,VWAP 15MA,VWAP 30MA,Volume 3MA,Volume 7MA,Volume 15MA,Volume 30MA,Value 3MA,Value 7MA,Value 15MA,Value 30MA,#Trades 3MA,#Trades 7MA,#Trades 15MA,#Trades 30MA,Target 3D,Target 7D,Target 15D,Target 30D
0,2020-01-01,1276.10,1280.00,1270.60,1279.00,1278.60,1276.64,2503.3,1084.00,1836849,2.345001e+09,46625,1,0,0,1,1,2020,2,1,1,1,1278.60,1278.60,1278.60,1278.60,1276.64,1276.64,1276.64,1276.64,1836849,1836849,1836849,1836849,2345000988,2345000988,2345000988,2345000988,46625,46625,46625,46625,1240.95,1282.70,1240.85,1240.60
1,2020-01-02,1279.00,1288.00,1279.00,1286.00,1286.75,1284.56,2503.3,1084.00,3068583,3.941792e+09,104570,1,0,0,2,1,2020,3,2,1,1,1282.68,1282.68,1282.68,1282.68,1280.60,1280.60,1280.60,1280.60,2452716,2452716,2452716,2452716,3143396262,3143396262,3143396262,3143396262,75597,75597,75597,75597,1260.60,1286.00,1244.85,1249.00
2,2020-01-03,1282.20,1285.00,1263.60,1268.50,1268.40,1270.48,2503.3,1084.00,5427775,6.895886e+09,157066,0,0,0,3,1,2020,4,3,1,1,1277.92,1277.92,1277.92,1277.92,1277.23,1277.23,1277.23,1277.23,3444402,3444402,3444402,3444402,4394226092,4394226092,4394226092,4394226092,102753,102753,102753,102753,1257.30,1289.50,1244.55,1241.40
3,2020-01-06,1260.00,1261.80,1236.00,1240.25,1240.95,1247.24,2503.3,1084.00,5445093,6.791348e+09,155007,0,0,0,6,1,2020,0,6,1,3,1265.37,1268.68,1268.68,1268.68,1267.43,1269.73,1269.73,1269.73,4647150,3944575,3944575,3944575,5876341707,4993506527,4993506527,4993506527,138881,115817,115817,115817,1271.40,1284.25,1213.20,1219.35
4,2020-01-07,1258.90,1271.45,1252.25,1261.00,1260.60,1261.48,2503.3,1084.00,7362247,9.287302e+09,189026,1,0,0,7,1,2020,1,7,1,1,1256.65,1267.06,1267.06,1267.06,1259.73,1268.08,1268.08,1268.08,6078371,4628109,4628109,4628109,7658178376,5852265530,5852265530,5852265530,167033,130458,130458,130458,1282.70,1287.65,1223.20,1217.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1004,2023-12-22,1683.60,1685.90,1667.10,1668.70,1670.85,1675.74,1757.5,1460.25,24289425,4.070271e+10,450284,0,0,0,22,12,2023,4,356,4,1,1671.52,1661.41,1646.06,1582.67,1670.28,1660.89,1645.06,1581.69,19900101,23959521,20674469,16842909,33252816061,39779977233,34019236236,26928124028,417161,361824,361372,308156,1705.25,NaN,NaN,NaN
1005,2023-12-26,1673.25,1685.95,1668.55,1684.10,1682.45,1678.46,1757.5,1460.25,9022928,1.514458e+10,243596,1,0,0,26,12,2023,1,360,4,4,1680.00,1666.02,1650.93,1589.04,1675.69,1664.81,1650.81,1588.08,17202324,22736199,19587249,16910087,28817261735,37795582215,32339953679,27085677731,359931,347304,348166,310241,1709.25,NaN,NaN,NaN
1006,2023-12-27,1681.50,1706.50,1678.60,1702.10,1703.30,1695.87,1757.5,1460.25,13504539,2.290188e+10,321373,1,0,0,27,12,2023,2,361,4,1,1685.53,1672.70,1656.24,1595.81,1683.36,1670.60,1655.46,1594.63,15605630,14748481,18746524,17294389,26249724747,24651270134,31035709963,27750333500,338417,333095,343117,318093,NaN,NaN,NaN,NaN
1007,2023-12-28,1709.30,1721.40,1702.00,1708.85,1705.25,1711.26,1757.5,1460.25,22038235,3.771305e+10,423062,0,0,0,28,12,2023,3,362,4,1,1697.00,1679.78,1661.40,1603.03,1695.20,1678.24,1661.05,1601.98,14855234,16610032,18928638,17685565,25253170191,27905665883,31455274908,28495473480,329343,356645,348588,321565,NaN,NaN,NaN,NaN
